<a href="https://colab.research.google.com/github/pedrambayat/MIC_prediction/blob/main/alphafold_fasta_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from huggingface_hub import hf_hub_download
file_path = hf_hub_download(
    repo_id="pedbb/mic_prediction",
    filename="baseline.csv",
    repo_type="dataset",
    token=None
)

df = pd.read_csv(file_path)
df.head()

baseline.csv: 0.00B [00:00, ?B/s]

,Unnamed: 0,ID,COMPLEXITY,SEQUENCE,SYNTHESIS TYPE,MIC,TARGET ID,SEQUENCE LENGTH,MOLECULAR WEIGHT,CHARGE,CHARGE DENSITY,ISOELECTRIC POINT,INSTABILITY INDEX,AROMATICITY,ALIPHATIC INDEX,BOWMAN INDEX,HYDROPHOBIC RATIO,is_AMP
0,0,1,2,NLVSGLIEARKYLEQLHRKLKNCKVENREVPPGFTALIKTLRKCKII,1,14.500,0,47.0,5462.54,6.963,0.001275,10.592285,25.882979,0.042553,114.042553,1.857234,0.404255,1
1,1,8,0,KVVVKWVVKVVK,2,100.000,0,12.0,1410.83,3.995,0.002832,11.156738,-19.258333,0.083333,169.166667,-0.700833,0.583333,0
2,2,11,0,RVKRVWPLVIRTVIAGYNLYRAIKKK,1,2.795,0,26.0,3141.85,7.993,0.002544,11.793945,8.011538,0.115385,127.307692,1.681538,0.423077,1
3,3,37,0,GMASKAGAIAGKIAKVALKAL,1,12.500,0,21.0,1969.45,3.995,0.002028,11.156738,-4.814286,0.000000,121.428571,-0.760000,0.619048,1
4,4,38,0,KAAAKWAAKAAK,2,6.250,0,12.0,1214.46,3.995,0.003290,11.156738,-3.358333,0.083333,58.333333,0.600000,0.583333,1


Isolate sequences of < 50 AAs long with complexity 0

In [ ]:
# Filter conditions
filtered = df[
    (df["SEQUENCE LENGTH"] < 50) &
    (df["COMPLEXITY"] == 0)
]

# View results
print(filtered["SEQUENCE"])

fasta_path = "/content/drive/MyDrive/CIS520FinalProjectData/filtered_peptides.fasta"

with open(fasta_path, "w") as f:
    for idx, row in filtered.iterrows():
        seq_id = row["ID"]              # or use row.name, or any unique identifier
        sequence = row["SEQUENCE"]

        # Write FASTA entry
        f.write(f">{seq_id}\n{sequence}\n")

print("FASTA file written to:", fasta_path)

1                           KVVVKWVVKVVK
2             RVKRVWPLVIRTVIAGYNLYRAIKKK
3                  GMASKAGAIAGKIAKVALKAL
4                           KAAAKWAAKAAK
5                                   KLLK
                      ...               
25295          WAGSAAIGAALPSVVGAFQKKKKKK
25296          WLGSALKIGAKLLPSVVGLFKKKKQ
25297                            FFWRRVK
25304    ALWKTLLKKVGKVAGKAVLNAVTNMANQNEQ
25305      ALWKTLLKKVGKVAGKAVLNAVTNMANQN
Name: SEQUENCE, Length: 24637, dtype: object
FASTA file written to: /content/drive/MyDrive/CIS520FinalProjectData/filtered_peptides.fasta


In [ ]:
alphafold_results_path = input("/content/drive/MyDrive/CIS520FinalProjectData/af_csvs")
print(f"AlphaFold results path set to: {alphafold_results_path}")

/content/drive/MyDrive/CIS520FinalProjectData/af_csvs
AlphaFold results path set to: 


In [ ]:
alphafold_results_path = "/content/drive/MyDrive/CIS520FinalProjectData/AlphaFold_Data"
print(f"AlphaFold results path set to: {alphafold_results_path}")

AlphaFold results path set to: /content/drive/MyDrive/CIS520FinalProjectData/AlphaFold_Data


In [ ]:
import os

processed_sequence_ids = []

# List all files in the alphafold_results_path directory
if os.path.exists(alphafold_results_path):
    for filename in os.listdir(alphafold_results_path):
        # Check if the file ends with '.result.zip'
        if filename.endswith('.result.zip'):
            # Extract the ID part of the filename
            id_str = filename.replace('.result.zip', '')
            try:
                # Convert the ID to an integer and add to the list
                processed_sequence_ids.append(int(id_str))
            except ValueError:
                # Handle cases where the ID is not a valid integer
                print(f"Warning: Could not parse ID from filename: {filename}")

print(f"Found {len(processed_sequence_ids)} processed sequence IDs.")
print(f"First 5 processed IDs: {processed_sequence_ids[:5]}")

Found 522 processed sequence IDs.
First 5 processed IDs: [7775, 7776, 7783, 7785, 7794]


In [ ]:
filtered_file_path = "/content/drive/MyDrive/CIS520FinalProjectData/filtered_peptides_trial_2.fasta"

# Read the FASTA file and parse it into a DataFrame
ids = []
sequences = []

with open(filtered_file_path, "r") as f:
    current_id = None
    current_sequence = []
    for line in f:
        line = line.strip()
        if line.startswith('>') and len(line) > 1:
            if current_id is not None:
                ids.append(int(current_id))
                sequences.append("".join(current_sequence))
            current_id = line[1:]
            current_sequence = []
        elif current_id is not None:
            current_sequence.append(line)
    # Add the last sequence
    if current_id is not None:
        ids.append(int(current_id))
        sequences.append("".join(current_sequence))

filtered = pd.DataFrame({'ID': ids, 'SEQUENCE': sequences})

unprocessed_peptides_df = filtered[~filtered['ID'].isin(processed_sequence_ids)]

print(f"Shape of unprocessed_peptides_df: {unprocessed_peptides_df.shape}")
print("Head of unprocessed_peptides_df:")
print(unprocessed_peptides_df.head())

Shape of unprocessed_peptides_df: (22780, 2)
Head of unprocessed_peptides_df:
   ID                    SEQUENCE
0   8                KVVVKWVVKVVK
1  11  RVKRVWPLVIRTVIAGYNLYRAIKKK
2  37       GMASKAGAIAGKIAKVALKAL
3  38                KAAAKWAAKAAK
4  72                KGGGKWGGKGGK


In [ ]:
unprocessed_peptides_df['SEQUENCE LENGTH'] = unprocessed_peptides_df['SEQUENCE'].apply(len)
unprocessed_peptides_df = unprocessed_peptides_df[unprocessed_peptides_df['SEQUENCE LENGTH'] >= 10]

print(f"Shape of unprocessed_peptides_df after filtering by length: {unprocessed_peptides_df.shape}")
print("Head of unprocessed_peptides_df after filtering by length:")
print(unprocessed_peptides_df.head())

Shape of unprocessed_peptides_df after filtering by length: (21312, 3)
Head of unprocessed_peptides_df after filtering by length:
   ID                    SEQUENCE  SEQUENCE LENGTH
0   8                KVVVKWVVKVVK               12
1  11  RVKRVWPLVIRTVIAGYNLYRAIKKK               26
2  37       GMASKAGAIAGKIAKVALKAL               21
3  38                KAAAKWAAKAAK               12
4  72                KGGGKWGGKGGK               12


In [ ]:
output_fasta_path = "/content/drive/MyDrive/CIS520FinalProjectData/filtered_peptides_trial_4.fasta"

with open(output_fasta_path, "w") as f:
    for idx, row in unprocessed_peptides_df.iterrows():
        seq_id = row["ID"]
        sequence = row["SEQUENCE"]
        f.write(f">{seq_id}\n{sequence}\n")

print("New FASTA file created at:", output_fasta_path)

New FASTA file created at: /content/drive/MyDrive/CIS520FinalProjectData/filtered_peptides_trial_4.fasta
